In [3]:


from nltk.ccg import chart, lexicon


lex = lexicon.parseLexicon('''
       :- S, NP, N, VP
    
        Det :: NP/N
        Pro :: NP
         Modal :: S\\NP/VP
    
         TV :: VP/NP
         DTV :: TV/NP
    
         the => Det
    
         that => Det
         a => Det
         the => Det
         that => NP
    
         I => Pro
         you => Pro
         we => Pro
    
         chef => N
         cake => N
         children => N
         dough => N
    
         will => Modal
         should => Modal
         might => Modal
         must => Modal
    
         and => var\\.,var/.,var
    
         to => VP[to]/VP
    
         without => (VP\\VP)/VP[ing]
    
         be => TV
         cook => TV
         eat => TV
    
         cooking => VP[ing]/NP
    
         give => DTV
    
         is => (S\\NP)/NP
        prefer => (S\\NP)/NP
    
        which => (N\\N)/(S/NP)
    
         persuade => (VP/VP[to])/NP
        ''')


<ipython-input-3-473d878506aa>:4: DeprecationWarning: 
  Function parseLexicon() has been deprecated.  Use fromstring()
  instead.
  lex = lexicon.parseLexicon('''


In [126]:
from collections import namedtuple

In [ ]:
# 1. hook $! #"%$	  0   7 hook   
# 2. For all labels  7- 	 :
# hole $) #"%$	  0  7 hole   0  +
# hole   
# 3. lzt $! #"%$   0   7 lzt   0  % lzt   
# 4. eq $! #"%$   0  7 ' *)   0  + *)  +
#  hook   0  7 hole ) 9"%$   - 
# where ' stands for transitive closure.



In [129]:
import numpy as np

In [222]:
RMRS = namedtuple("RMRS", ["hook", "hole", "rels", "hcons", "equalities"])

Pred = namedtuple('Pred', ['name', 'args'])

EPS = namedtuple('EPS', ['handle', 'pred'])

def build_matrix(eq):
    var = set()
    for a, b in eq:
        var |= {a, b}
    
    var = sorted(list(var))
    
    matrix = np.zeros((len(var), len(var)))

    for i, a in enumerate(var):
        for j, b in enumerate(var):
            if (a, b) in eq or (b, a) in eq:
                matrix[i,j] = 1
    return matrix, var
    
def matrix_to_eq(mat, var):
    for i, a in enumerate(var):
        for j, b in enumerate(var):
            if mat[i,j] == 1 and a != b and j > i:
                yield (a, b)
                
    
def tr(eq):
    mat, var = build_matrix(eq)
    
    
    mat = tr_helper(mat, var)
    
    return list(matrix_to_eq(mat, var))
    
def tr_helper(mat, var):
    updated = False
    for i in range(len(var)):
        for j in range(len(var)):
            if mat[i, j] == 1:
                continue
            else:
                for k in range(len(var)):
                    if mat[i, k] == 1 and mat[k, j] == 1:
                        mat[i, j] = 1
                        updated = True
    if updated:
        return tr_helper(mat, var)
    else:
        return mat
    
    
eq = [('x1', 'x2'), ('x2', 'x3'), ('x4', 'x5')]
mat, var = build_matrix(eq)
assert(all([e in eq for e in matrix_to_eq(mat, var)]))
eq2 = [('x1', 'x2'), ('x2', 'x3'), ('x4', 'x5'), ('x1', 'x3')]
assert(all([e in eq2 for e in tr(eq)]))
    
    
hnum = 0

def get_hnum():
    global hnum
    h = f'h{hnum}'
    hnum += 1
    return h

xnum = 0
def get_x():
    global xnum
    x = f'x{xnum}'
    xnum += 1
    return x

def op(a1, a2, label):
    hook = a2.hook
    
    slots = {key: a1.hole.get(key, {}) | a2.hole.get(key, {}) for key in a1.hole.keys() | a2.hole.keys() if key != label}
    
    
    new_rels = a1.rels + a2.rels 
    equalities = tr(a1.equalities | a2.equalities | {(a1.hook[0], a2.hole[label][0]), (a1.hook[1], a2.hole[label][1])})
    hcons = a1.hcons | a2.hcons
    
    return RMRS(hook, slots, new_rels, hcons, equalities)


cat = RMRS(['h1', 'x2'], {}, [EPS('h1', Pred('cat', ['x2']))], set(), set())
every = RMRS(['h2', 'x3'], {'spec':['h2', 'x3']}, [EPS('h3', Pred('every', ['x3']))], set(), set())

op(cat, every, 'spec')

RMRS(hook=['h2', 'x3'], hole={}, rels=[EPS(handle='h1', pred=Pred(name='cat', args=['x2'])), EPS(handle='h3', pred=Pred(name='every', args=['x3']))], hcons=set(), equalities=[('h1', 'h2'), ('x2', 'x3')])

In [223]:
get_hnum()

'h0'

In [224]:
h = get_hnum()
h2 = get_hnum()
x = get_x()
every = RMRS([get_hnum(), x], {'spec':[h, x]}, [EPS(h, Pred('every', [x, h2, get_hnum()]))], {(h, h2)}, set())
h = get_hnum()
x = get_x()
dog = RMRS([h, x], {}, [EPS(h, Pred('dog', [x]))], set(), set())

op(dog, every, 'spec')

RMRS(hook=['h3', 'x0'], hole={}, rels=[EPS(handle='h5', pred=Pred(name='dog', args=['x1'])), EPS(handle='h1', pred=Pred(name='every', args=['x0', 'h2', 'h4']))], hcons={('h1', 'h2')}, equalities=[('h1', 'h5'), ('x0', 'x1')])

In [226]:
h = get_hnum()
x = get_x()
white = RMRS([h, x], {'mod':[h, x]}, [EPS(h, Pred('white', [x]))], set(), set())

op(dog, white, 'mod')

RMRS(hook=['h8', 'x2'], hole={}, rels=[EPS(handle='h5', pred=Pred(name='dog', args=['x1'])), EPS(handle='h8', pred=Pred(name='white', args=['x2']))], hcons=set(), equalities=[('h5', 'h8'), ('x1', 'x2')])

In [201]:
op(cat, every, 'spec')

x2 x3


RMRS(hook=['x3'], hole={}, rels=[Pred(name='cat', args=['x2']), Pred(name='every', args=['x3'])], hcons=set(), equalities=[('x2', 'x3')])

In [174]:
eq = [('x1', 'x2'), ('x3', 'x2'), ('x4', 'x5')]
mat, var = build_matrix(eq)
mat, var

(array([[0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.]]),
 ['x1', 'x2', 'x3', 'x4', 'x5'])

In [175]:
tr_helper(mat, var)

array([[1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [176]:
list(matrix_to_eq(mat, var))

[('x1', 'x2'), ('x1', 'x3'), ('x2', 'x3'), ('x4', 'x5')]

In [177]:
tr(eq)

[('x1', 'x2'), ('x1', 'x3'), ('x2', 'x3'), ('x4', 'x5')]

In [121]:
import spacy

nlp = spacy.load("en_core_web_sm")
sents = nlp(u'a mug should be on it')

In [122]:
for chunk in sents.noun_chunks:
    print(chunk)

a mug
it


In [123]:
for t in sents:
    print(t, t.dep, t.tag)

a 415 15267657372422890137
mug 429 15794550382381185553
should 405 16235386156175103506
be 8206900633647566924 14200088355797579614
on 443 1292078113972184607
it 439 13656873538139661788


In [124]:
sents.to_json()

{'text': 'a mug should be on it',
 'ents': [],
 'sents': [{'start': 0, 'end': 21}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 1,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 1},
  {'id': 1,
   'start': 2,
   'end': 5,
   'pos': 'PROPN',
   'tag': 'NNP',
   'dep': 'nsubj',
   'head': 3},
  {'id': 2,
   'start': 6,
   'end': 12,
   'pos': 'VERB',
   'tag': 'MD',
   'dep': 'aux',
   'head': 3},
  {'id': 3,
   'start': 13,
   'end': 15,
   'pos': 'AUX',
   'tag': 'VB',
   'dep': 'ROOT',
   'head': 3},
  {'id': 4,
   'start': 16,
   'end': 18,
   'pos': 'ADP',
   'tag': 'IN',
   'dep': 'prep',
   'head': 3},
  {'id': 5,
   'start': 19,
   'end': 21,
   'pos': 'PRON',
   'tag': 'PRP',
   'dep': 'pobj',
   'head': 4}]}

In [66]:
def get_dets(sents):
    json_sent = sents.to_json()
    for token in json_sent['tokens']:
        if token['dep'] == 'det':
            yield token
            
def get_dps(sents):
    json_sent = sents.to_json()
    dets = get_dets(sents)
    for det in dets:
        head = json_sent['tokens'][det['head']]
        adj = json_sent['tokens'][det['id']+1:head['id']]
        out = tuple([det] + adj + [head])
        yield out
    
def get_root(sents):
    json_sent = sents.to_json()
    for token in json_sent['tokens']:
        if token['dep'] == 'ROOT':
            return token
        
def get_dp_word(dp, sents):
    phrase = " ".join([get_word(sents, t) for t in dp])
    return phrase

def get_word(sents, token):
    return str(sents)[token['start']:token['end']]

In [96]:
from collections import MutableSet

class OrderedSet(MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)

In [108]:
from collections import defaultdict
from itertools import product
import sys
import copy

class Grammar():
    
    def __init__(self, lexicon, rules, functions):
        self.lexicon = lexicon
        self.rules = rules
        self.functions = functions
        
    def parse(self, sentence):
        words = sentence.split()
        
        trace = defaultdict(list)
        n = len(words) + 1
        
        for i in range(1, n):
            word = words[i-1]
            trace[(i-1, i)] = [[(syntax, semantics), word] for syntax, semantics, in self.lexicon[word]]
        
        for j in range(2, n):
            for i in range(j-1, -1, -1):
                for k in range(i+1, j):
                    for c1, c2 in product(trace[(i, k)], trace[(k, j)]):
                        
                        for c3 in self.use_rules(c1[0], c2[0]):
                            trace[(i, j)].append([c3, f"{c1[1]} {c2[1]}"])
                            
        return trace[(0, n-1)]
                 
        
    def early(self, sentence):
        words = sentence.split()
        
        S = [OrderedSet() for i in range(len(words)+1)]
        
        S[0].add((('<P>', ['.', '<S>']), 0))
        
        for i in range(0, len(words)):
            for state, idx in S[i]:
                if not finished(state):
                    if is_non_terminal(next_element_of(state)):
                        predictor(state, i, grammar, S)
                    else:
                        scanner(state, i, words, S)
                else:
                    pass
    
    def use_rules(self, c1, c2):
        for left, right, parent, app_order in self.rules:
            if left == c1[0] and right == c2[0]:
                sem = [c1[1], c2[1]]
                yield (parent, f"{sem[app_order[0]]}({sem[app_order[1]]})")
        
    def sem(self, lf):
        """Interpret, as Python code, the root of a logical form
        generated by this grammar."""
        # Import all of the user's functions into the namespace to
        # help with the interpretation of the logical forms.
        grammar = sys.modules[__name__]
        for key, val in list(self.functions.items()):
            setattr(grammar, key, val)        
        return eval(lf[0][1]) # Interpret just the root node's semantics. 
        
def finished(state):
    return state[1][-1] == '.'
        
def next_element_of(state):
    for i in range(len(state[1])):
        if state[1][i] == '.':
            return state[1][i+1]
        
def is_non_terminal(element):
    return '<' in element and '>' in element

def new_state(state):
    p, r = state
    r = copy.deepcopy(r)
    r.insert(0, '.')

def predictor(state, i, grammar, S):
    B = next_element_of(state)
    for rule in [(p, r) for p, r in grammar if p == B]:
        
        S[i+1].append((p, new_state(r)))

def scanner(state, i, words):
    a = next_element_of(state)
    if a in 
        
g = Grammar(gold_lexicon, rules, functions)
# p = g.parse("minus two plus three")
# print(p)
# for lf in p:
#     print(g.sem(lf))

print(finished(('<P>', ['<S>', '.'])))
print(finished(('<P>', ['.', '<S>'])))

True
False


In [107]:
r = [1, 2]
r.insert('a', 0)
r

TypeError: 'str' object cannot be interpreted as an integer

In [104]:
<P> ::= <S>      # the start rule
<S> ::= <S> "+" <M> | <M>
<M> ::= <M> "*" <T> | <T>
<T> ::= "1" | "2" | "3" | "4"

SyntaxError: invalid syntax (<ipython-input-104-143a9e8212dc>, line 1)

In [100]:
rules = [
    ('<P>', ['<S>']),
    ('<S>', ['<S>', '+', '<M>']),
    ('<S>', ['M']),
    ('<M>', ['<M>', '*', '<T>']),
    ('<M>', ['<T>']),
    ('<T>', ['1']),
    ('<T>', ['2']),
    ('<T>', ['3']),
    ('<T>', ['4'])
]

In [68]:
gold_lexicon = {
    'one':   [('N', '1')],
    'two':   [('N', '2')],
    'three': [('N', '3')],
    'four':  [('N', '4')],
    'five':  [('N', '5')],
    'six':   [('N', '6')],
    'seven': [('N', '7')],
    'eight': [('N', '8')],
    'nine':  [('N', '9')],
    'plus':  [('R', 'add')],
    'minus': [('R', 'subtract'), ('U', 'neg')],
    'times': [('R', 'multiply')],
}

In [69]:
rules = [
    ['B', 'N', 'N', (0,1)], 
    ['U', 'N', 'N', (0,1)], 
    ['N', 'R', 'B', (1,0)]
]

In [70]:
functions = {
    'neg': (lambda x : -x),
    'add': (lambda x : (lambda y : x + y)),
    'subtract': (lambda x : (lambda y : x - y)),
    'multiply': (lambda x : (lambda y : x * y))
}



In [67]:
for t in get_dps(sents):
    print(get_dp_word(t, sents))

a green cup
the table


In [49]:
sents.to_json()

{'text': 'There should be a green cup on the table',
 'ents': [],
 'sents': [{'start': 0, 'end': 40}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'pos': 'PRON',
   'tag': 'EX',
   'dep': 'expl',
   'head': 2},
  {'id': 1,
   'start': 6,
   'end': 12,
   'pos': 'VERB',
   'tag': 'MD',
   'dep': 'aux',
   'head': 2},
  {'id': 2,
   'start': 13,
   'end': 15,
   'pos': 'AUX',
   'tag': 'VB',
   'dep': 'ROOT',
   'head': 2},
  {'id': 3,
   'start': 16,
   'end': 17,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 5},
  {'id': 4,
   'start': 18,
   'end': 23,
   'pos': 'ADJ',
   'tag': 'JJ',
   'dep': 'amod',
   'head': 5},
  {'id': 5,
   'start': 24,
   'end': 27,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'attr',
   'head': 2},
  {'id': 6,
   'start': 28,
   'end': 30,
   'pos': 'ADP',
   'tag': 'IN',
   'dep': 'prep',
   'head': 5},
  {'id': 7,
   'start': 31,
   'end': 34,
   'pos': 'DET',
   'tag': 'DT',
   'dep': 'det',
   'head': 8},
  {'id': 8,
   'start': 35

In [ ]:
lexicon = {''}

In [1]:
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)

NameError: name 'chart' is not defined

In [2]:
for parse in parser.parse("you prefer that cake".split()):  # doctest: +SKIP
    chart.printCCGDerivation(parse)
    break

NameError: name 'parser' is not defined

In [4]:
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)
for parse in parser.parse("I should give you the cake".split()):  # doctest: +SKIP
    chart.printCCGDerivation(parse)
    break

 I     should         give      you   the    cake
 NP  ((S\NP)/VP)  ((VP/NP)/NP)  NP   (NP/N)   N
                 ------------------->
                       (VP/NP)
                                    -------------->
                                          NP
                 --------------------------------->
                                VP
    ---------------------------------------------->
                        (S\NP)
--------------------------------------------------<
                        S


In [18]:
from nltk.parse import load_parser
from nltk.sem.drt import DrtParser
parser = load_parser('grammar.fcfg', trace=1, logic_parser=DrtParser())
for tree in parser.parse('there is a table in the centre'.split()):
     print(tree.label()['SEM'].simplify())

|.t.i.a.t.i.t.c.|
|[-] . . . . . .| [0:1] 'there'
|. [-] . . . . .| [1:2] 'is'
|. . [-] . . . .| [2:3] 'a'
|. . . [-] . . .| [3:4] 'table'
|. . . . [-] . .| [4:5] 'in'
|. . . . . [-] .| [5:6] 'the'
|. . . . . . [-]| [6:7] 'centre'
|[-> . . . . . .| [0:1] VP[NUM=?n] -> 'there' * BE[NUM=?n] {}
|[-] . . . . . .| [0:1] THERE[] -> 'there' *
|. [-] . . . . .| [1:2] BE[NUM='sg'] -> 'is' *
|[---] . . . . .| [0:2] VP[NUM='sg'] -> 'there' BE[NUM='sg'] *
|[---> . . . . .| [0:2] S[SEM=?np] -> VP[NUM=?n] * NPP[NUM=?n, SEM=?np] {?n: 'sg'}
|. . [-] . . . .| [2:3] Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] -> 'a' *
|. . [-> . . . .| [2:3] NP[NUM=?n, SEM=<?det(?np)>] -> Det[NUM=?n, SEM=?det] * N[NUM=?n, SEM=?np] {?det: <DrtLambdaExpression \P Q.(([x],[]) + P(x) + Q(x))>, ?n: 'sg'}
|. . . [-] . . .| [3:4] N[NUM='sg', SEM=<\x.([],[Table(x)])>] -> 'table' *
|. . [---] . . .| [2:4] NP[NUM='sg', SEM=<\Q.(([x],[Table(x)]) + Q(x))>] -> Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] N[NUM='sg', SEM

In [22]:
g = parser.grammar()

In [25]:
g

<Grammar with 17 productions>

In [7]:
for tree in parser.parse('The table is 1.8m wide and 0.8m long'.split()):
     print(tree.label()['SEM'].simplify())

ValueError: Grammar does not cover some of the input words: "'The', '1.8m', 'wide', 'and', '0.8m', 'long'".

In [10]:
from ProbRobNLP.probrob import *
from nltk.sem.drt import *

In [11]:

    
class WorldModel():
    
    def __init__(self):
        pass

    

    
class InterpretationSystem():
    
    def __init__(self):
        self.dialogue_history = []
        self.dialogue_referents = []
        self.parser = load_parser('grammar.fcfg', trace=0, logic_parser=DrtParser())
        
        
    def read_sentence(self, sentence: str):
        for tree in self.parser.parse(sentence.split()):
             sem = tree.label()['SEM'].simplify()
        
        return sem
    
    def read_DRS(self, drs: DRS):
        
        refs = [r.name for r in drs.refs]
        preds = [parse_cond(cond) for cond in drs.conds]
        return DRSState(preds, refs)
            

            
                
def parse_cond(cond):
    if isinstance(cond, DrtApplicationExpression):
        pred = cond.pred.variable.name
#         print(pred)
        args = [parse_cond(x) for x in cond.args]
#         print(args)

        return Predicate(pred, args)

#         if pred in ['table', 'cup']:
            
#             obj = (pred, args)
#             return obj
            
#         elif pred in ['vector']:
#             vector = Vector3D(*args)
#             return vector
#         elif pred in ['on']:
#             return (pred, args)
        
    if isinstance(cond, DrtIndividualVariableExpression) or isinstance(cond, DrtConstantExpression):
        return cond.variable.name

In [25]:

class Predicate(object):

    def __init__(self, name, args, op=None):
        self.name = name
        self.args = args
        self.valency = not(op == 'not')
        self.op = op

    def negate(self):
        op = 'not' if self.valency is True else None
        return Predicate(self.name, self.args, op)

    def __str__(self):
        return f"{self.name}({','.join(map(str, self.args))})"

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.name == other.name and all([arg1 == arg2 for arg1, arg2 in zip(self.args, other.args)]) and len(self.args) == len(other.args)

    def __ne__(self, other):
        return not(self.__eq__(other))

    def to_dexpr(self):
        
        return DrtExpression.fromstring(self.__str__())
        
    def toPRS(self):
        if self.name == "on":
            arg1 = get_prs(self.args[1])
            return OnConstraint(arg1)
        elif self.name == "vector":
            return Vector3D(*map(get_prs, self.args))
        
        
def get_prs(predicate):
    if isinstance(predicate, Predicate):
        return predicate.toPRS()
    else:
        return predicate
    
class DRSState(object):
    def __init__(self, predicates, referents):
        self.referents = referents
        self.predicates = predicates

    def get_predicates(self, arg):
        return [pred for pred in self.predicates if arg in pred.args]
    
    def get_first_place_predicates(self, arg):
        return [pred for pred in self.predicates if arg == pred.args[0]]

    def _predicate_holds(self, predicate):
        return predicate in self.predicates

    def predicate_holds(self, predicate_name, args):
        predicate = Predicate(predicate_name, args)

    def __eq__(self, other):
        for predicate in self.predicates:
            if predicate not in other.predicates:
                return False
        for predicate in other.predicates:
            if predicate not in self.predicates:
                return False
        return True

    def asDRS(self):
        dexpr = [pred.to_dexpr() for pred in self.predicates]
        return DRS(self.referents, dexpr)
    
    def print(self):
        drs = self.asDRS()
        drs.pretty_print()

        
    def toPRS(self):
        for referent in self.referents:
            type_ = None
            constraints = []
            for predicate in self.get_first_place_predicates(referent):
                if predicate.name in ['Table', 'Cup']:
                    type_ = predicate.name
                else:
                    constraints.append(predicate.toPRS())
            print(Entity(type_, referent, constraints))
            
    

In [26]:
preds = [Predicate('Table', ['x']), Predicate('on', ['x', Predicate('vector', [0, 0, 0])])]
d = DRSState(preds, ['x'])

In [27]:
d.toPRS()

x = Table on Vector3D(0, 0, 0)


In [28]:
s = InterpretationSystem()
sem = s.read_sentence("there is a table on the centre")
s.read_DRS(sem).toPRS()


x = Table on Vector3D(0, 0, 0)


In [114]:
sem.pretty_print()

 _____________________ 
| x                   |
|---------------------|
| table(x)            |
| on(x,vector(0,0,0)) |
|_____________________|


In [108]:
sem.predicates()
fol = sem.fol()

In [111]:
sem.refs[0].name

'x'

In [89]:
t.args[1].args[0].variable.name

'0'

In [49]:
fol_sem = sem.fol()

In [50]:
and_expr = fol_sem.term

In [51]:
f = and_expr.first

In [52]:
s= and_expr.second

In [53]:
s.args

[<DrtIndividualVariableExpression x>, <ApplicationExpression vector(0,0,0)>]